In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("../new_data.csv")
df.English = df.English.str.replace('\n', ' ')
df=df.dropna(axis=0)
df.head()

,Id,Enunciado,Tópico,Contexto,English
0,994,Meu primeiro programa\nEscreva um programa que...,Ambientação,Outros,"""My first show Write a program that prints the..."
1,996,Impressão de caracteres na tela\nEscreva um pr...,Ambientação,Outros,Screen character printing Write a program tha...
2,999,Impressão de caracteres na tela (Bart Simpson)...,Ambientação,Outros,Screen Character Printing (Bart Simpson) Writ...
3,1000,Operadores aritméticos\nQual o valor de X para...,Ambientação,Matemático,Arithmetic Operators What is the value of X s...
4,578,Operadores aritméticos \nQual o valor de Y par...,Ambientação,Matemático,Arithmetic Operators What is the value of Y s...


In [3]:
contador=0
lista=[]
while(contador<len(df['English'])):
    if(df['Contexto'][contador]=='Outros'):
        lista+=[contador]
    contador+=1
df=df.drop(lista)

In [4]:
import spacy
nlp = spacy.load('en')

In [5]:
master=[]
for elemento in df['English']:
    doc=nlp(elemento)
    lista=[]
    for token in doc:
        if(token.text not in nlp.Defaults.stop_words and token.lemma_ not in nlp.Defaults.stop_words and token.text.isalpha()):
            lista+=[token.lemma_]
    master+=[' '.join(lista)]

In [6]:
df['tokens']=master

In [7]:
X = df['tokens'].values
y = df['Contexto'].values

In [17]:
class GloveVectorizer:
    def __init__(self):
        # load in pre-trained word vectors
        print('Loading word vectors...')
        word2vec = {}
        embedding = []
        idx2word = []
        with open('../../../../cursos/pln/novo_pln_deep-learning/nlp_class2/myCodes/glove/glove.6B.300d.txt') as f:
            # is just a space-separated text file in the format:
            # word vec[0] vec[1] vec[2] ...
            for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:], dtype='float32')
                word2vec[word] = vec
                embedding.append(vec)
                idx2word.append(word)
        print('Found %s word vectors.' % len(word2vec))

        # save for later
        self.word2vec = word2vec
        self.embedding = np.array(embedding)
        self.word2idx = {v:k for k,v in enumerate(idx2word)}
        self.V, self.D = self.embedding.shape

    def fit(self, data):
        pass

    def transform(self, data):
        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            for word in tokens:
                if word in self.word2vec:
                    vec = self.word2vec[word]
                    vecs.append(vec)
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount += 1
            n += 1
        print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [18]:
from gensim.models import KeyedVectors
class Word2VecVectorizer:
    def __init__(self):
        print("Loading in word vectors...")
        self.word_vectors = KeyedVectors.load_word2vec_format('../../../../cursos/pln/novo_pln_deep-learning/nlp_class2/myCodes/google/GoogleNews-vectors-negative300.bin',binary=True)
        print("Finished loading in word vectors")

    def fit(self, data):
        pass

    def transform(self, data):
        # determine the dimensionality of vectors
        v = self.word_vectors.get_vector('king')
        self.D = v.shape[0]

        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.split()
            vecs = []
            m = 0
            for word in tokens:
                try:
                    # throws KeyError if word not found
                    vec = self.word_vectors.get_vector(word)
                    vecs.append(vec)
                    m += 1
                except KeyError:
                    pass
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount += 1
            n += 1
        print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X


    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [19]:
vectorizer = GloveVectorizer()
# vectorizer = Word2VecVectorizer()

Loading word vectors...
Found 400000 word vectors.


In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [21]:
from sklearn.preprocessing import LabelEncoder
X = df['tokens'].values
y = df['Contexto'].values
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [22]:
from sklearn.model_selection import StratifiedKFold
vetor_X_train=[]
vetor_y_train=[]
vetor_X_test=[]
vetor_y_test=[]
split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for train_index, val_index in split.split(X, y):
    vetor_X_train+=[X[train_index]]
    vetor_X_test+=[X[val_index]]
    vetor_y_train+=[y[train_index]]
    vetor_y_test+=[y[val_index]]

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


In [23]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from keras import layers

In [24]:
def baseline_model():
    
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(300,)))
    model.add(Dropout(p=0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p=0.25))
    model.add(layers.Dense(32, activation='relu'))
    model.add(Dropout(p=0.25))
    model.add(layers.Dense(23, activation='sigmoid'))


    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [25]:
contador=0
predictions=[]
while(contador<len(vetor_X_train)):
    X_train=vetor_X_train[contador]
    y_train=vetor_y_train[contador]
    X_test=vetor_X_test[contador]
    y_test=vetor_y_test[contador]
    
    encoded_y = encoder.transform(y_train)
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_y)
        
    X_train = vectorizer.fit_transform(X_train)
    estimator = KerasClassifier(build_fn=baseline_model, epochs=150, batch_size=20)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    predictions+=[y_pred]

    contador+=1

Numer of samples with no words found: 0 / 887


/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.25)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.25)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.25)`
  if __name__ == '__main__':


Epoch 1/150
887/887 [==============================] - 1s 926us/step - loss: 0.4354 - accuracy: 0.8234
Epoch 2/150
887/887 [==============================] - 0s 121us/step - loss: 0.2057 - accuracy: 0.9438
Epoch 3/150
887/887 [==============================] - 0s 104us/step - loss: 0.1762 - accuracy: 0.9538
Epoch 4/150
887/887 [==============================] - 0s 105us/step - loss: 0.1570 - accuracy: 0.9564
Epoch 5/150
887/887 [==============================] - 0s 129us/step - loss: 0.1411 - accuracy: 0.9617
Epoch 6/150
887/887 [==============================] - 0s 129us/step - loss: 0.1301 - accuracy: 0.9622
Epoch 7/150
887/887 [==============================] - 0s 128us/step - loss: 0.1202 - accuracy: 0.9639
Epoch 8/150
887/887 [==============================] - 0s 111us/step - loss: 0.1094 - accuracy: 0.9680
Epoch 9/150
887/887 [==============================] - 0s 115us/step - loss: 0.1001 - accuracy: 0.9697
Epoch 10/150
887/887 [==============================] - 0s 118us/step - l

887/887 [==============================] - 0s 118us/step - loss: 0.0107 - accuracy: 0.9963
Epoch 80/150
887/887 [==============================] - 0s 122us/step - loss: 0.0085 - accuracy: 0.9971
Epoch 81/150
887/887 [==============================] - 0s 123us/step - loss: 0.0080 - accuracy: 0.9975
Epoch 82/150
887/887 [==============================] - 0s 136us/step - loss: 0.0091 - accuracy: 0.9967
Epoch 83/150
887/887 [==============================] - 0s 136us/step - loss: 0.0095 - accuracy: 0.9965
Epoch 84/150
887/887 [==============================] - 0s 129us/step - loss: 0.0084 - accuracy: 0.9970
Epoch 85/150
887/887 [==============================] - 0s 136us/step - loss: 0.0061 - accuracy: 0.9980
Epoch 86/150
887/887 [==============================] - 0s 176us/step - loss: 0.0081 - accuracy: 0.9971
Epoch 87/150
887/887 [==============================] - 0s 150us/step - loss: 0.0080 - accuracy: 0.9972
Epoch 88/150
887/887 [==============================] - 0s 161us/step - loss:

887/887 [==============================] - 0s 109us/step - loss: 0.1290 - accuracy: 0.9633
Epoch 7/150
887/887 [==============================] - 0s 143us/step - loss: 0.1162 - accuracy: 0.9671
Epoch 8/150
887/887 [==============================] - 0s 179us/step - loss: 0.1093 - accuracy: 0.9677
Epoch 9/150
887/887 [==============================] - 0s 141us/step - loss: 0.1026 - accuracy: 0.9695
Epoch 10/150
887/887 [==============================] - 0s 105us/step - loss: 0.0921 - accuracy: 0.9714
Epoch 11/150
887/887 [==============================] - 0s 95us/step - loss: 0.0874 - accuracy: 0.9718
Epoch 12/150
887/887 [==============================] - 0s 105us/step - loss: 0.0795 - accuracy: 0.9743
Epoch 13/150
887/887 [==============================] - 0s 130us/step - loss: 0.0760 - accuracy: 0.9740
Epoch 14/150
887/887 [==============================] - 0s 134us/step - loss: 0.0708 - accuracy: 0.9752
Epoch 15/150
887/887 [==============================] - 0s 120us/step - loss: 0.0

...

888/888 [==============================] - 0s 106us/step - loss: 0.0046 - accuracy: 0.9980
Epoch 119/150
888/888 [==============================] - 0s 122us/step - loss: 0.0045 - accuracy: 0.9985
Epoch 120/150
888/888 [==============================] - 0s 135us/step - loss: 0.0039 - accuracy: 0.9988
Epoch 121/150
888/888 [==============================] - 0s 123us/step - loss: 0.0039 - accuracy: 0.9987
Epoch 122/150
888/888 [==============================] - 0s 125us/step - loss: 0.0046 - accuracy: 0.9982
Epoch 123/150
888/888 [==============================] - 0s 112us/step - loss: 0.0056 - accuracy: 0.9979
Epoch 124/150
888/888 [==============================] - 0s 116us/step - loss: 0.0047 - accuracy: 0.9986
Epoch 125/150
888/888 [==============================] - 0s 122us/step - loss: 0.0052 - accuracy: 0.9984
Epoch 126/150
888/888 [==============================] - 0s 122us/step - loss: 0.0045 - accuracy: 0.9987
Epoch 127/150
888/888 [==============================] - 0s 110us/ste

888/888 [==============================] - 0s 97us/step - loss: 0.0178 - accuracy: 0.9933
Epoch 46/150
888/888 [==============================] - 0s 98us/step - loss: 0.0160 - accuracy: 0.9946
Epoch 47/150
888/888 [==============================] - 0s 100us/step - loss: 0.0161 - accuracy: 0.9943
Epoch 48/150
888/888 [==============================] - 0s 92us/step - loss: 0.0158 - accuracy: 0.9945
Epoch 49/150
888/888 [==============================] - 0s 92us/step - loss: 0.0158 - accuracy: 0.9944
Epoch 50/150
888/888 [==============================] - 0s 91us/step - loss: 0.0144 - accuracy: 0.9946
Epoch 51/150
888/888 [==============================] - 0s 95us/step - loss: 0.0141 - accuracy: 0.9949
Epoch 52/150
888/888 [==============================] - 0s 88us/step - loss: 0.0157 - accuracy: 0.9946
Epoch 53/150
888/888 [==============================] - 0s 98us/step - loss: 0.0131 - accuracy: 0.9951
Epoch 54/150
888/888 [==============================] - 0s 89us/step - loss: 0.0142 -

Epoch 124/150
888/888 [==============================] - 0s 110us/step - loss: 0.0048 - accuracy: 0.9983
Epoch 125/150
888/888 [==============================] - 0s 113us/step - loss: 0.0047 - accuracy: 0.9980
Epoch 126/150
888/888 [==============================] - 0s 121us/step - loss: 0.0039 - accuracy: 0.9987
Epoch 127/150
888/888 [==============================] - 0s 108us/step - loss: 0.0039 - accuracy: 0.9987
Epoch 128/150
888/888 [==============================] - 0s 125us/step - loss: 0.0048 - accuracy: 0.9984
Epoch 129/150
888/888 [==============================] - 0s 112us/step - loss: 0.0038 - accuracy: 0.9988
Epoch 130/150
888/888 [==============================] - 0s 113us/step - loss: 0.0038 - accuracy: 0.9989
Epoch 131/150
888/888 [==============================] - 0s 113us/step - loss: 0.0033 - accuracy: 0.9990
Epoch 132/150
888/888 [==============================] - 0s 117us/step - loss: 0.0051 - accuracy: 0.9983
Epoch 133/150
888/888 [==============================] 

888/888 [==============================] - 0s 112us/step - loss: 0.0153 - accuracy: 0.9945
Epoch 52/150
888/888 [==============================] - 0s 117us/step - loss: 0.0149 - accuracy: 0.9947
Epoch 53/150
888/888 [==============================] - 0s 129us/step - loss: 0.0137 - accuracy: 0.9955
Epoch 54/150
888/888 [==============================] - 0s 137us/step - loss: 0.0121 - accuracy: 0.9955
Epoch 55/150
888/888 [==============================] - 0s 111us/step - loss: 0.0135 - accuracy: 0.9954
Epoch 56/150
888/888 [==============================] - 0s 100us/step - loss: 0.0117 - accuracy: 0.9960
Epoch 57/150
888/888 [==============================] - 0s 114us/step - loss: 0.0096 - accuracy: 0.9970
Epoch 58/150
888/888 [==============================] - 0s 112us/step - loss: 0.0112 - accuracy: 0.9959
Epoch 59/150
888/888 [==============================] - 0s 113us/step - loss: 0.0122 - accuracy: 0.9957
Epoch 60/150
888/888 [==============================] - 0s 101us/step - loss:

888/888 [==============================] - 0s 98us/step - loss: 0.0046 - accuracy: 0.9985
Epoch 130/150
888/888 [==============================] - 0s 95us/step - loss: 0.0043 - accuracy: 0.9983
Epoch 131/150
888/888 [==============================] - 0s 88us/step - loss: 0.0039 - accuracy: 0.9988
Epoch 132/150
888/888 [==============================] - 0s 91us/step - loss: 0.0035 - accuracy: 0.9990
Epoch 133/150
888/888 [==============================] - 0s 92us/step - loss: 0.0043 - accuracy: 0.9982
Epoch 134/150
888/888 [==============================] - 0s 91us/step - loss: 0.0040 - accuracy: 0.9987
Epoch 135/150
888/888 [==============================] - 0s 92us/step - loss: 0.0043 - accuracy: 0.9984
Epoch 136/150
888/888 [==============================] - 0s 92us/step - loss: 0.0037 - accuracy: 0.9989
Epoch 137/150
888/888 [==============================] - 0s 89us/step - loss: 0.0032 - accuracy: 0.9991
Epoch 138/150
888/888 [==============================] - 0s 101us/step - loss:

In [26]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in encoder.inverse_transform(lista1):
        y_pred+=[elemento1]


In [27]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [28]:
from sklearn import metrics

In [29]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

[[ 31   2   0   0   0   0   0   0   0   0   0   1   1   0   0   0   0   0
    0   0   0   0   0]
 [  3 103   0   0   1   1   0   0   0   0   1   1   1   0   0   0   0   3
    0   3   1   0   2]
 [  0   0   4   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0  15   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   1]
 [  0   1   0   0  17   1   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   0  73   1   0   0   1   0   0   3   0   0   0   0   0
    0   1   0   0   0]
 [  0   0   0   0   0   0  42   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0  30   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  34   0   0   1   0   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   0   0   0   1   0   9   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0   0   0   0 

In [30]:
print(metrics.classification_report(y_test_sla,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.86      0.89      0.87        35
      Comercial       0.91      0.86      0.88       120
  Computacional       1.00      0.67      0.80         6
        Consumo       1.00      0.94      0.97        16
    Data e hora       0.94      0.81      0.87        21
        Escolar       0.95      0.92      0.94        79
        Esporte       0.98      1.00      0.99        42
Filmes e séries       0.97      1.00      0.98        30
         Física       0.92      0.94      0.93        36
      Geografia       0.82      0.82      0.82        11
        Imposto       0.80      0.80      0.80         5
          Jogos       0.95      0.92      0.93        96
     Matemático       0.95      0.98      0.97       261
  Meio ambiente       0.94      0.89      0.91        18
       Pesquisa       1.00      0.89      0.94        18
         Pessoa       0.95      0.91      0.93        22
      População       0.89    

In [31]:
print(metrics.accuracy_score(y_test_sla,y_pred))

0.9279918864097363


In [32]:
np.savetxt('ytrue_GLOVE+ANN_Context-Others', y_test_sla, fmt='%s')
np.savetxt('ypred_GLOVE+ANN_Context-Others', y_pred, fmt='%s')

In [33]:
def noOthers(df):
    contador=0
    lista=[]
    while(contador<len(df['English'])):
        if(df['Contexto'][contador]=='Outros'):
            lista+=[contador]
        contador+=1
    return df.drop(lista)

In [34]:
dfs_train = []
dfs_test = []
for i in range(0, 10):
    df_train = pd.read_csv('../dadosParafrase/treino'+str(i+1)+'.csv')
    df_train=noOthers(df_train)
    dfs_train.append(df_train)
    df_test = pd.read_csv('../dadosParafrase/teste'+str(i+1)+'.csv')
    df_test=noOthers(df_test)
    dfs_test.append(df_test)

In [35]:
def baseline_model():
    
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(300,)))
    model.add(Dropout(p=0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p=0.25))
    model.add(layers.Dense(32, activation='relu'))
    model.add(Dropout(p=0.25))
    model.add(layers.Dense(23, activation='sigmoid'))


    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [36]:
contador=0
predictions=[]
vetor_y_test=[]
while(contador<len(dfs_train)):
    X_train=dfs_train[contador]['master'].values
    y_train=dfs_train[contador]['Contexto'].values
    X_test=dfs_test[contador]['master'].values
    vetor_y_test+=[dfs_test[contador]['Contexto'].values]
    
    
    encoded_y = encoder.transform(y_train)
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_y)
        
    X_train = vectorizer.fit_transform(X_train)
    estimator = KerasClassifier(build_fn=baseline_model, epochs=150, batch_size=20)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    predictions+=[y_pred]

    contador+=1

Numer of samples with no words found: 0 / 2870


/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.25)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.25)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.25)`
  if __name__ == '__main__':


Epoch 1/150
2870/2870 [==============================] - 1s 204us/step - loss: 0.2910 - accuracy: 0.8983
Epoch 2/150
2870/2870 [==============================] - 0s 114us/step - loss: 0.1612 - accuracy: 0.9564
Epoch 3/150
2870/2870 [==============================] - 0s 111us/step - loss: 0.1196 - accuracy: 0.9617
Epoch 4/150
2870/2870 [==============================] - 0s 119us/step - loss: 0.0940 - accuracy: 0.9673
Epoch 5/150
2870/2870 [==============================] - 0s 122us/step - loss: 0.0763 - accuracy: 0.9735
Epoch 6/150
2870/2870 [==============================] - 0s 111us/step - loss: 0.0623 - accuracy: 0.9774
Epoch 7/150
2870/2870 [==============================] - 0s 117us/step - loss: 0.0518 - accuracy: 0.9813
Epoch 8/150
2870/2870 [==============================] - 0s 110us/step - loss: 0.0457 - accuracy: 0.9834
Epoch 9/150
2870/2870 [==============================] - 0s 103us/step - loss: 0.0386 - accuracy: 0.9865
Epoch 10/150
2870/2870 [==============================]

2870/2870 [==============================] - 0s 121us/step - loss: 0.0025 - accuracy: 0.9993
Epoch 79/150
2870/2870 [==============================] - 0s 117us/step - loss: 0.0023 - accuracy: 0.9993
Epoch 80/150
2870/2870 [==============================] - 0s 117us/step - loss: 0.0030 - accuracy: 0.9990
Epoch 81/150
2870/2870 [==============================] - 0s 87us/step - loss: 0.0034 - accuracy: 0.9990
Epoch 82/150
2870/2870 [==============================] - 0s 96us/step - loss: 0.0028 - accuracy: 0.9990
Epoch 83/150
2870/2870 [==============================] - 0s 98us/step - loss: 0.0020 - accuracy: 0.9993
Epoch 84/150
2870/2870 [==============================] - 0s 91us/step - loss: 0.0028 - accuracy: 0.9993
Epoch 85/150
2870/2870 [==============================] - 0s 122us/step - loss: 0.0025 - accuracy: 0.9992
Epoch 86/150
2870/2870 [==============================] - 0s 89us/step - loss: 0.0023 - accuracy: 0.9993
Epoch 87/150
2870/2870 [==============================] - 0s 110

2875/2875 [==============================] - 0s 122us/step - loss: 0.0934 - accuracy: 0.9688
Epoch 5/150
2875/2875 [==============================] - 0s 122us/step - loss: 0.0746 - accuracy: 0.9740
Epoch 6/150
2875/2875 [==============================] - 0s 122us/step - loss: 0.0601 - accuracy: 0.9790
Epoch 7/150
2875/2875 [==============================] - 0s 99us/step - loss: 0.0504 - accuracy: 0.9824
Epoch 8/150
2875/2875 [==============================] - 0s 77us/step - loss: 0.0431 - accuracy: 0.9848
Epoch 9/150
2875/2875 [==============================] - 0s 99us/step - loss: 0.0376 - accuracy: 0.9868
Epoch 10/150
2875/2875 [==============================] - 0s 93us/step - loss: 0.0335 - accuracy: 0.9886
Epoch 11/150
2875/2875 [==============================] - 0s 90us/step - loss: 0.0283 - accuracy: 0.9902
Epoch 12/150
2875/2875 [==============================] - 0s 79us/step - loss: 0.0264 - accuracy: 0.9908
Epoch 13/150
2875/2875 [==============================] - 0s 83us/step

...

2875/2875 [==============================] - 0s 102us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 100/150
2875/2875 [==============================] - 0s 108us/step - loss: 0.0016 - accuracy: 0.9996
Epoch 101/150
2875/2875 [==============================] - 0s 92us/step - loss: 0.0016 - accuracy: 0.9994
Epoch 102/150
2875/2875 [==============================] - 0s 109us/step - loss: 0.0019 - accuracy: 0.9993
Epoch 103/150
2875/2875 [==============================] - 0s 117us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 104/150
2875/2875 [==============================] - 0s 121us/step - loss: 0.0027 - accuracy: 0.9989
Epoch 105/150
2875/2875 [==============================] - 0s 111us/step - loss: 0.0020 - accuracy: 0.9994
Epoch 106/150
2875/2875 [==============================] - 0s 110us/step - loss: 0.0023 - accuracy: 0.9993
Epoch 107/150
2875/2875 [==============================] - 0s 121us/step - loss: 0.0024 - accuracy: 0.9993
Epoch 108/150
2875/2875 [===========================

2875/2875 [==============================] - 0s 101us/step - loss: 0.0093 - accuracy: 0.9970
Epoch 26/150
2875/2875 [==============================] - 0s 89us/step - loss: 0.0091 - accuracy: 0.9969
Epoch 27/150
2875/2875 [==============================] - 0s 89us/step - loss: 0.0083 - accuracy: 0.9972
Epoch 28/150
2875/2875 [==============================] - 0s 95us/step - loss: 0.0086 - accuracy: 0.9971
Epoch 29/150
2875/2875 [==============================] - 0s 106us/step - loss: 0.0077 - accuracy: 0.9974
Epoch 30/150
2875/2875 [==============================] - 0s 112us/step - loss: 0.0075 - accuracy: 0.9976
Epoch 31/150
2875/2875 [==============================] - 0s 124us/step - loss: 0.0072 - accuracy: 0.9975
Epoch 32/150
2875/2875 [==============================] - 0s 111us/step - loss: 0.0069 - accuracy: 0.9979
Epoch 33/150
2875/2875 [==============================] - 0s 97us/step - loss: 0.0061 - accuracy: 0.9977
Epoch 34/150
2875/2875 [==============================] - 0s 12

2875/2875 [==============================] - 0s 97us/step - loss: 0.0025 - accuracy: 0.9992
Epoch 103/150
2875/2875 [==============================] - 0s 103us/step - loss: 0.0023 - accuracy: 0.9992
Epoch 104/150
2875/2875 [==============================] - 0s 117us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 105/150
2875/2875 [==============================] - 0s 111us/step - loss: 0.0024 - accuracy: 0.9993
Epoch 106/150
2875/2875 [==============================] - 0s 105us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 107/150
2875/2875 [==============================] - 0s 106us/step - loss: 0.0027 - accuracy: 0.9991
Epoch 108/150
2875/2875 [==============================] - 0s 105us/step - loss: 0.0021 - accuracy: 0.9994
Epoch 109/150
2875/2875 [==============================] - 0s 99us/step - loss: 0.0019 - accuracy: 0.9993
Epoch 110/150
2875/2875 [==============================] - 0s 84us/step - loss: 0.0019 - accuracy: 0.9994
Epoch 111/150
2875/2875 [=============================

2866/2866 [==============================] - 0s 112us/step - loss: 0.0081 - accuracy: 0.9974
Epoch 29/150
2866/2866 [==============================] - 0s 120us/step - loss: 0.0080 - accuracy: 0.9975
Epoch 30/150
2866/2866 [==============================] - 0s 116us/step - loss: 0.0076 - accuracy: 0.9975
Epoch 31/150
2866/2866 [==============================] - 0s 117us/step - loss: 0.0079 - accuracy: 0.9973
Epoch 32/150
2866/2866 [==============================] - 0s 119us/step - loss: 0.0071 - accuracy: 0.9978
Epoch 33/150
2866/2866 [==============================] - 0s 107us/step - loss: 0.0070 - accuracy: 0.9977
Epoch 34/150
2866/2866 [==============================] - 0s 112us/step - loss: 0.0061 - accuracy: 0.9981
Epoch 35/150
2866/2866 [==============================] - 0s 113us/step - loss: 0.0061 - accuracy: 0.9980
Epoch 36/150
2866/2866 [==============================] - 0s 107us/step - loss: 0.0060 - accuracy: 0.9979
Epoch 37/150
2866/2866 [==============================] - 0

2866/2866 [==============================] - 0s 117us/step - loss: 0.0025 - accuracy: 0.9991
Epoch 106/150
2866/2866 [==============================] - 0s 97us/step - loss: 0.0025 - accuracy: 0.9992
Epoch 107/150
2866/2866 [==============================] - 0s 90us/step - loss: 0.0020 - accuracy: 0.9993
Epoch 108/150
2866/2866 [==============================] - 0s 93us/step - loss: 0.0022 - accuracy: 0.9992
Epoch 109/150
2866/2866 [==============================] - 0s 89us/step - loss: 0.0016 - accuracy: 0.9993
Epoch 110/150
2866/2866 [==============================] - 0s 87us/step - loss: 0.0019 - accuracy: 0.9994
Epoch 111/150
2866/2866 [==============================] - 0s 89us/step - loss: 0.0018 - accuracy: 0.9995
Epoch 112/150
2866/2866 [==============================] - 0s 102us/step - loss: 0.0022 - accuracy: 0.9994
Epoch 113/150
2866/2866 [==============================] - 0s 95us/step - loss: 0.0031 - accuracy: 0.9991
Epoch 114/150
2866/2866 [==============================] -

In [37]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in encoder.inverse_transform(lista1):
        y_pred+=[elemento1]

In [38]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [39]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

[[ 33   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  3 100   0   1   2   1   0   0   0   0   0   1   2   0   0   1   0   2
    0   4   0   0   3]
 [  0   0   5   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0  19   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   0  75   0   0   0   1   0   0   3   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0  42   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0  30   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  35   0   0   1   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   8   0   0   1   0   0   0   0   0
    0   1   0   0   0]
 [  0   1   0   0   0   0   0 

In [40]:
print(metrics.classification_report(y_test_sla,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.87      0.94      0.90        35
      Comercial       0.89      0.83      0.86       120
  Computacional       1.00      0.83      0.91         6
        Consumo       0.94      1.00      0.97        16
    Data e hora       0.79      0.90      0.84        21
        Escolar       0.97      0.95      0.96        79
        Esporte       0.93      1.00      0.97        42
Filmes e séries       0.94      1.00      0.97        30
         Física       0.95      0.97      0.96        36
      Geografia       0.73      0.73      0.73        11
        Imposto       1.00      0.80      0.89         5
          Jogos       0.95      0.93      0.94        96
     Matemático       0.97      0.97      0.97       261
  Meio ambiente       1.00      0.89      0.94        18
       Pesquisa       1.00      0.89      0.94        18
         Pessoa       0.91      0.91      0.91        22
      População       0.96    

In [43]:
print(metrics.accuracy_score(y_test_sla,y_pred))

0.9310344827586207


In [44]:
np.savetxt('ytrue_GLOVE+ANN_Context-Others+PAR', y_test_sla, fmt='%s')
np.savetxt('ypred_GLOVE+ANN_Context-Others+PAR', y_pred, fmt='%s')

In [37]:
y_true=[]
y_trueArq=open('./ytrue_ANN_Context-Others')
for linha in y_trueArq:
    y_true+=[linha]
y_pred=[]
y_predArq=open('./ypred_ANN_Context-Others')
for linha in y_predArq:
    y_pred+=[linha]
print(metrics.accuracy_score(y_true,y_pred))

0.8864097363083164


In [61]:
def baseline_model():
    
    model = Sequential()
    model.add(Dense(64, activation='tanh', input_shape=(300,)))
    model.add(Dropout(p=0.1))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(p=0.1))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(p=0.1))
    model.add(Dense(7, activation='softmax'))


    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [62]:
estimator = KerasClassifier(build_fn=baseline_model)

In [63]:
from sklearn.model_selection import GridSearchCV
paramaters={'epochs':[150,200,250],
            'batch_size':[10,20],
            'optimizer':['adam','rmsprop']}

In [64]:
X = df['tokens'].values
X = vectorizer.fit_transform(X)
y = df['category'].values
# encoder = LabelEncoder()
# encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(encoded_y)

Numer of samples with no words found: 0 / 4610


In [ ]:
grid=GridSearchCV(estimator=estimator,
                 param_grid=paramaters,
                 cv=10)
grid.fit(X,y)

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


Epoch 1/150
4149/4149 [==============================] - 1s 166us/step - loss: 1.4529 - accuracy: 0.4102
Epoch 2/150
4149/4149 [==============================] - 1s 142us/step - loss: 1.3049 - accuracy: 0.4784
Epoch 3/150
4149/4149 [==============================] - 1s 153us/step - loss: 1.2429 - accuracy: 0.5020
Epoch 4/150
4149/4149 [==============================] - 1s 146us/step - loss: 1.2135 - accuracy: 0.5170
Epoch 5/150
4149/4149 [==============================] - 1s 158us/step - loss: 1.1933 - accuracy: 0.5225
Epoch 6/150
4149/4149 [==============================] - 1s 145us/step - loss: 1.1670 - accuracy: 0.5396
Epoch 7/150
4149/4149 [==============================] - 1s 140us/step - loss: 1.1505 - accuracy: 0.5493
Epoch 8/150
4149/4149 [==============================] - 1s 133us/step - loss: 1.1513 - accuracy: 0.5466
Epoch 9/150
4149/4149 [==============================] - 1s 141us/step - loss: 1.1215 - accuracy: 0.5657
Epoch 10/150
4149/4149 [==============================]

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


Epoch 1/150
4149/4149 [==============================] - 1s 192us/step - loss: 1.4930 - accuracy: 0.4261
Epoch 2/150
4149/4149 [==============================] - 1s 168us/step - loss: 1.3089 - accuracy: 0.4992
Epoch 3/150
4149/4149 [==============================] - 1s 140us/step - loss: 1.2573 - accuracy: 0.5107
Epoch 4/150
4149/4149 [==============================] - 1s 179us/step - loss: 1.2265 - accuracy: 0.5271
Epoch 5/150
4149/4149 [==============================] - 1s 231us/step - loss: 1.2002 - accuracy: 0.5450
Epoch 6/150
4149/4149 [==============================] - 1s 225us/step - loss: 1.1726 - accuracy: 0.5490
Epoch 7/150
4149/4149 [==============================] - 1s 127us/step - loss: 1.1621 - accuracy: 0.5544
Epoch 8/150
4149/4149 [==============================] - 1s 153us/step - loss: 1.1455 - accuracy: 0.5592
Epoch 9/150
4149/4149 [==============================] - 0s 118us/step - loss: 1.1449 - accuracy: 0.5625
Epoch 10/150
4149/4149 [==============================]

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


...

4149/4149 [==============================] - 1s 182us/step - loss: 0.2177 - accuracy: 0.9231
Epoch 155/200
4149/4149 [==============================] - 1s 183us/step - loss: 0.2228 - accuracy: 0.9217
Epoch 156/200
4149/4149 [==============================] - 1s 166us/step - loss: 0.2198 - accuracy: 0.9272
Epoch 157/200
4149/4149 [==============================] - 1s 159us/step - loss: 0.2033 - accuracy: 0.9287
Epoch 158/200
4149/4149 [==============================] - 1s 157us/step - loss: 0.2245 - accuracy: 0.9171
Epoch 159/200
4149/4149 [==============================] - 1s 145us/step - loss: 0.2063 - accuracy: 0.9301
Epoch 160/200
4149/4149 [==============================] - 1s 155us/step - loss: 0.2080 - accuracy: 0.9289
Epoch 161/200
4149/4149 [==============================] - 1s 150us/step - loss: 0.2063 - accuracy: 0.9229
Epoch 162/200
4149/4149 [==============================] - 1s 153us/step - loss: 0.2075 - accuracy: 0.9301
Epoch 163/200
4149/4149 [==========================

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


Epoch 1/200
4149/4149 [==============================] - 3s 632us/step - loss: 1.3718 - accuracy: 0.4232
Epoch 2/200
4149/4149 [==============================] - 0s 115us/step - loss: 1.2293 - accuracy: 0.4796
Epoch 3/200
4149/4149 [==============================] - 0s 118us/step - loss: 1.1725 - accuracy: 0.5107
Epoch 4/200
4149/4149 [==============================] - 0s 119us/step - loss: 1.1349 - accuracy: 0.5300
Epoch 5/200
4149/4149 [==============================] - 0s 116us/step - loss: 1.1303 - accuracy: 0.5377
Epoch 6/200
4149/4149 [==============================] - 1s 121us/step - loss: 1.0942 - accuracy: 0.5546
Epoch 7/200
4149/4149 [==============================] - 1s 153us/step - loss: 1.0923 - accuracy: 0.5522
Epoch 8/200
4149/4149 [==============================] - 1s 132us/step - loss: 1.0754 - accuracy: 0.5536
Epoch 9/200
4149/4149 [==============================] - 1s 134us/step - loss: 1.0642 - accuracy: 0.5678
Epoch 10/200
4149/4149 [==============================]

4149/4149 [==============================] - 1s 154us/step - loss: 0.1984 - accuracy: 0.9262
Epoch 155/200
4149/4149 [==============================] - 1s 137us/step - loss: 0.1990 - accuracy: 0.9289
Epoch 156/200
4149/4149 [==============================] - 1s 156us/step - loss: 0.1779 - accuracy: 0.9383
Epoch 157/200
4149/4149 [==============================] - 1s 133us/step - loss: 0.1968 - accuracy: 0.9270
Epoch 158/200
4149/4149 [==============================] - 1s 151us/step - loss: 0.2022 - accuracy: 0.9299
Epoch 159/200
4149/4149 [==============================] - 1s 134us/step - loss: 0.2043 - accuracy: 0.9253
Epoch 160/200
4149/4149 [==============================] - 1s 163us/step - loss: 0.2068 - accuracy: 0.9315
Epoch 161/200
4149/4149 [==============================] - 1s 137us/step - loss: 0.1903 - accuracy: 0.9340
Epoch 162/200
4149/4149 [==============================] - 1s 152us/step - loss: 0.1981 - accuracy: 0.9287
Epoch 163/200
4149/4149 [==========================

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)